In [1]:
!dir .\train

 Volume in drive C is OS
 Volume Serial Number is A479-8D11

 Directory of C:\Users\bazif\Desktop\Project\train

23-02-2022  12:05    <DIR>          .
23-02-2022  12:05    <DIR>          ..
04-03-2022  15:42    <DIR>          BCC
21-02-2022  22:33    <DIR>          BKL
04-03-2022  15:44    <DIR>          MEL
               0 File(s)              0 bytes
               5 Dir(s)  563,724,804,096 bytes free


In [2]:
# import tensorflow as tf
# from keras.layers import Dense
# from keras.models import Model
# model =tf.keras.applications.ResNet50(
#     include_top=False,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling="max",
# )
# dense = Dense(3,activation = "softmax",kernel_initializer=tf.keras.initializers.HeNormal())(model.layers[-1].output)
# transfer_model = Model(inputs = model.input,outputs=dense)
# transfer_model.save("ResnetBase.hdf5")

In [3]:
import tensorflow as tf
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model
from keras.layers import Flatten
import matplotlib.pyplot as plt
model =tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
    pooling="max",
    classes=3,
    classifier_activation="softmax",
)
# model.summary()
transfer_model = None
dense = Dense(3,activation = "softmax",kernel_initializer=tf.keras.initializers.HeNormal())(model.layers[-1].output)
transfer_model = Model(inputs = model.input,outputs=dense)

transfer_model.summary()    
        

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

In [4]:
model= transfer_model

In [5]:
import os
dirs = os.listdir(".\\train")
for each in dirs:
    print(each+ " " +str(len(os.listdir(".\\train\\" + each))))

BCC 2624
BKL 2624
MEL 2624


In [6]:
import numpy as np
import time
import uuid
import cv2
def preprocess_img(img):
    img=img.astype(np.uint8)
    equ=cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    clahe = cv2.createCLAHE(clipLimit=3., tileGridSize=(8,8))
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    l2 = clahe.apply(l)
    lab = cv2.merge((l2,a,b))
    img2 = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return equ.astype(np.float32)
def test():
    print("hi\n")
from keras.callbacks import ModelCheckpoint
filepath="EF2-{epoch:02d}-auc_{val_auc:2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_auc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.compile(optimizer="adam",loss = "categorical_crossentropy",metrics=["accuracy",tf.keras.metrics.AUC(multi_label = True)])
classifier=transfer_model
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        preprocessing_function=preprocess_img,
        horizontal_flip=True,
#         preprocessing_function=preprocess_img,
        validation_split=0.2)
total_images = 25331

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_img)

training_set= train_datagen.flow_from_directory(
        './train',
        subset="training",
        target_size=(224, 224),
        batch_size=32)
validatin_set = train_datagen.flow_from_directory(
        "./train",
        subset = "validation",
        target_size=(224, 224),
        batch_size=32)


# # Class weights
# from sklearn.model_selection import train_test_split
# from sklearn.utils import class_weight

# from collections import Counter
# class_weights = class_weight.compute_class_weight('balanced',
#                                                   classes=np.unique(training_set.classes), 
#                                                   y=training_set.classes) 

# unique_class_weights = np.unique(training_set.classes)
# class_weights_dict   = { unique_class_weights[i]: w for i,w in enumerate(class_weights) }

# print('\nCLASS WEIGHTS: {}\n'.format(class_weights))
# print(np.unique(training_set.classes))
# print(training_set.classes)
# print(unique_class_weights)
# print(Counter(training_set.classes).keys())   # equals to list(set(x))
# print(Counter(training_set.classes).values()) # counts the elements' frequency

Found 6300 images belonging to 3 classes.
Found 1572 images belonging to 3 classes.


In [ ]:
hist=classifier.fit(
        training_set,
        steps_per_epoch=6300//32,
        epochs=20,
        validation_data=validatin_set,
        validation_steps=1572//32,callbacks=callbacks_list)
print(hist.history)

Epoch 1/20
196/196 [==============================] - 128s 575ms/step - loss: 1.2623 - accuracy: 0.6589 - auc: 0.8108 - val_loss: 1.1706 - val_accuracy: 0.3329 - val_auc: 0.4763

Epoch 00001: val_auc improved from -inf to 0.47625, saving model to EF2-01-auc_0.476250.hdf5


C:\Users\bazif\anaconda3\envs\TensorflowGPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/20
196/196 [==============================] - 112s 571ms/step - loss: 0.6644 - accuracy: 0.7518 - auc: 0.8965 - val_loss: 1.1676 - val_accuracy: 0.3195 - val_auc: 0.5262

Epoch 00002: val_auc improved from 0.47625 to 0.52616, saving model to EF2-02-auc_0.526157.hdf5
Epoch 3/20
196/196 [==============================] - 114s 579ms/step - loss: 0.7018 - accuracy: 0.7486 - auc: 0.8952 - val_loss: 1.1700 - val_accuracy: 0.3335 - val_auc: 0.5424

Epoch 00003: val_auc improved from 0.52616 to 0.54235, saving model to EF2-03-auc_0.542352.hdf5
Epoch 4/20
196/196 [==============================] - 111s 563ms/step - loss: 0.4867 - accuracy: 0.8074 - auc: 0.9365 - val_loss: 1.1680 - val_accuracy: 0.3329 - val_auc: 0.5299

Epoch 00004: val_auc did not improve from 0.54235
Epoch 5/20
196/196 [==============================] - 111s 566ms/step - loss: 0.4085 - accuracy: 0.8376 - auc: 0.9540 - val_loss: 1.1258 - val_accuracy: 0.2966 - val_auc: 0.4903

Epoch 00005: val_auc did not improve from 

In [1]:
classifier.save("trained_effb_0.hdf5")

NameError: name 'classifier' is not defined

In [ ]:
from keras.models import load_model
saved = load_model("eficientnetb0-09-acc_0.43.hdf5")
mdl = Model(inputs=saved.input,outputs=saved.output)
from keras.optimizers import Adam
mdl.compile(optimizer="adam",loss = "categorical_crossentropy",metrics=["accuracy"])
mdl.fit_generator(
        training_set,
        steps_per_epoch=8377//16,
        epochs=10,
        validation_data=validatin_set,
        validation_steps=2092//16,callbacks=callbacks_list,class_weight=class_weights_dict)